In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import numpy as np
import os
from selenium import webdriver

In [7]:
##scrape nasa mars news site

URL = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"

driver = webdriver.Firefox()

driver.get(URL)
html = driver.page_source

soup = bs(html, "html.parser")

news_title = soup.find("div", class_="content_title").text

news_p = soup.find("div", class_ = "article_teaser_body").text
print(news_title, news_p)

The MarCO Mission Comes to an End The pair of briefcase-sized satellites made history when they sailed past Mars in 2019.


In [ ]:
##requests to get images and full-size image urls
img_URL = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
driver = webdriver.Firefox()
driver.get(img_URL)
html = driver.page_source
soup = bs(html, "html.parser")
img_path = soup.find("footer").a.attrs
feat_img_url = "https://www.jpl.nasa.gov" + img_path["data-fancybox-href"]
feat_img_url

In [ ]:
##get mars weather
weather_URL = "https://twitter.com/marswxreport?lang=en"
html = requests.get(weather_URL).text
soup = bs(html, "html.parser")

mars_weather = soup.find("p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text
mars_weather

In [4]:
##get & tabulate mars facts
facts_URL = "https://space-facts.com/mars/"
facts_df = pd.read_html(facts_URL)[0]
facts_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [ ]:
mars_facts_df.columns = ["Description", "Values"]
mars_facts_df.set_index("Description")

In [6]:
##get mars hemispheres images, save to dict "title":"name", "image_url":"image"

In [13]:
hemi_URL = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
driver.get(hemi_URL)
html = driver.page_source
soup = bs(html, "html.parser")

hemi_name = [e.text.strip("Enhanced") for e in soup.find_all("h3")]
hemi_name



['Cerberus Hemisphere ',
 'Schiaparelli Hemisphere ',
 'Syrtis Major Hemisphere ',
 'Valles Marineris Hemisphere ']

In [14]:
img_path = [e.a["href"] for e in soup.find_all("div", class_="description")]
img_link = []
for e in range(len(img_path)):
    img_link.append("https://astrogeology.usgs.gov" + img_path[e])
img_link

['https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced']

In [10]:
img_url = []
for e in range(len(img_link)):
    driver.get(img_link[e])
    html = driver.page_source
    soup = bs(html, "html.parser")
    
    
    img_url.append("https://astrogeology.usgs.gov" + soup.find_all("img", class_="wide-image")[0]["src"])
    
img_url

['https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg']

['Downlo',
 'FGDC: xml metadat',
 'Related Products',
 'Valles Marineris Hemisphere U',
 'Syrtis Major Hemisphere U',
 'Syrtis Major Hemisphere ',
 'Schiaparelli Hemisphere U',
 'Schiaparelli Hemisphere ']

In [16]:
hemi_img = []
for e in range(len(hemi_name)):
    hemi_dict = {}
    hemi_dict["title"] = hemi_name[e]
    hemi_dict["url"] = img_url[e]
    
    hemi_img.append(hemi_dict)
    
hemi_img

[{'title': 'Cerberus Hemisphere ',
  'url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere ',
  'url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere ',
  'url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere ',
  'url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]